#### Before You Start


cd .devcontainer </br>
run docker-compose up

In [ ]:
! pip install psycopg2
! pip install sqlalchemy


In [ ]:
import psycopg2  #import of the psycopg2 python library
import pandas as pd #import of the pandas python library
import pandas.io.sql as psql

##No transaction is started when commands are executed and no commit() or rollback() is required. 
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

In [ ]:
try:
    # Connect to the postgreSQL server with username, and password credentials
    con = psycopg2.connect(user = "postgres",
                                  password = "postgres",
                                  host = "localhost",
                                  port = "5432")
    
    con.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT);
    print("Connected Successfully to PostgreSQL server!!")
    
    # Obtain a DB Cursor to perform database operations
    cursor = con.cursor();
except (Exception, psycopg2.Error) as error :
     print ("Error while connecting to PostgreSQL", error)


#### View table in python

In [ ]:
import pandas as pd

df = pd.read_csv('lap_time.csv')
print(df.head())

### Create tables in relational database

In [ ]:
try:
    #table_name variable
    create_all_tables_query = '''

    DROP TABLE IF EXISTS lap_time, pit_stop, result, race, driver, constructor, circuit, season CASCADE;

    CREATE TABLE IF NOT EXISTS circuit (
        circuit_id INT PRIMARY KEY,
        -- circuit_ref omitted
        name TEXT,
        location TEXT,
        country TEXT,
        lat float,
        long float,
        alt INT,
        url TEXT
    );

    CREATE TABLE IF NOT EXISTS constructor (
        constructor_id INT PRIMARY KEY,
        -- removing constructor_ref for normalisation purposes
        name TEXT,
        nationality TEXT,
        url TEXT
    );

    CREATE TABLE IF NOT EXISTS driver (
        driver_id INT PRIMARY KEY,
        -- removing driver_ref and number for normalisation purposes
        code TEXT,
        first_name TEXT, --no longer forename
        last_name TEXT, --no longer surname
        dob DATE,
        nationality TEXT,
        url TEXT
    );

    CREATE TABLE IF NOT EXISTS lap_time (
        race_id INT,
        driver_id INT REFERENCES driver,
        lap INT,
        position INT,
        time_in_milliseconds INT,
        PRIMARY KEY(race_id, driver_id, lap)
    );

    CREATE TABLE IF NOT EXISTS pit_stop (
        race_id INT,
        driver_id INT,
        stop INT,
        lap INT,
        time_in_miliseconds INT,
        PRIMARY KEY(race_id, driver_id, lap)
    );

    CREATE TABLE IF NOT EXISTS race (
        race_id INT PRIMARY KEY,
        year INT,
        round INT,
        circuit_id INT REFERENCES circuit,
        name TEXT,
        date DATE,
        url TEXT
    );

    CREATE TABLE IF NOT EXISTS result (
        result_id INT PRIMARY KEY,
        race_id INT REFERENCES race,
        driver_id INT REFERENCES driver,
        constructor_id INT REFERENCES constructor,
        -- number and grid omitted
        position INT, -- use positionOrder instead of position/positionText
        points INT,
        laps INT,
        -- omit time, unclean
        time_in_milliseconds INT,
        fastest_lap INT,
        rank INT,
        fastest_lap_time_in_milliseconds INT,
        fastest_lap_speed float
    );

    CREATE TABLE IF NOT EXISTS season (
        year INT PRIMARY KEY,
        url TEXT
    );

    '''


    #Execute this command (SQL Query)
    cursor.execute(create_all_tables_query)
    
    # Make the changes to the database persistent
    con.commit()
    print("All tables created successfully in PostgreSQL ")
except (Exception, psycopg2.Error) as error:
    # if it exits with an exception the transaction is rolled back.
    con.rollback()
    print("Error While Creating the DB: ",error)

### Create many-to-many tables

In [ ]:
try:
    #table_name variable
    create_all_tables_query = '''

    DROP TABLE IF EXISTS race_result, constructor_result CASCADE;

    CREATE TABLE IF NOT EXISTS race_result (
        race_id INT REFERENCES race,
        result_id INT REFERENCES result, 
        PRIMARY KEY (race_id, result_id)
    );

    CREATE TABLE IF NOT EXISTS constructor_result (
        constructor_id INT REFERENCES constructor,
        result_id INT REFERENCES result,
        PRIMARY KEY (constructor_id, result_id)
    );
    '''


    #Execute this command (SQL Query)
    cursor.execute(create_all_tables_query)
    
    # Make the changes to the database persistent
    con.commit()
    print("All tables created successfully in PostgreSQL ")
except (Exception, psycopg2.Error) as error:
    # if it exits with an exception the transaction is rolled back.
    con.rollback()
    print("Error While Creating the DB: ",error)

### Check table creation

In [ ]:
# [information_schema.tables] keep listing of every table being managed by Postgres for a particular database.
# specifying the tabel_schema to 'public' to only list tables that you create.
cursor.execute("""SELECT table_name 
                  FROM information_schema.tables 
                  WHERE table_schema = 'public'  
               """)

for table in cursor.fetchall():
    print(table)

### Read all CSVs and insert into postgres (requires some waiting)

In [ ]:
import pandas as pd
from sqlalchemy import create_engine

# List of CSV files and corresponding table names
csv_files = [
    ("circuit.csv", "circuit"),
    ("constructor.csv", "constructor"),
    ("driver.csv", "driver"),
    ("lap_time.csv", "lap_time"),
    ("pit_stop.csv", "pit_stop"),
    ("race.csv", "race"),
    ("result.csv", "result"),
    ("season.csv", "season")
]

# Connect to the PostgreSQL database
user = "postgres"
password = "postgres"
host = "localhost"
port = "5432"
database = "postgres"
engine = create_engine(f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}")

# Loop through the list of CSV files and import them into the PostgreSQL tables
for csv_file, table_name in csv_files:
    try:
        # Load CSV file into DataFrame
        # df = pd.read_csv(csv_file)
        df = pd.read_csv(csv_file, encoding="latin1")

        # Write the DataFrame to the PostgreSQL table
        df.to_sql(table_name, con=engine, if_exists="append", index=False, method="multi", chunksize=100)
        # df.to_sql(table_name, con=engine, if_exists="append", index=False)
        print(f"Data from {csv_file} successfully imported into {table_name}!")
    except Exception as e:
        print(f"An error occurred while importing {csv_file} into {table_name}: {e}")


### Populate many-to-many tables with records

In [ ]:
csv_file = 'result.csv'
table_name = 'race_result'

try:
        # Load CSV file into DataFrame
        # df = pd.read_csv(csv_file)
        df_result = pd.read_csv('result.csv', encoding="latin1")
        df_race_result = df_result[['race_id', 'result_id']]
        df_constructor_result = df_result[['constructor_id', 'result_id']]

        # Write the DataFrame to the PostgreSQL table
        df_race_result.to_sql('race_result', con=engine, if_exists="append", index=False, method="multi", chunksize=100)
        # df.to_sql(table_name, con=engine, if_exists="append", index=False)
        print(f"Data from {csv_file} successfully imported into {table_name}!")
except Exception as e:
    print(f"An error occurred while importing {csv_file} into {table_name}: {e}")


In [ ]:
csv_file = 'result.csv'
table_name = 'constructor_result'

try:
        # Load CSV file into DataFrame
        # df = pd.read_csv(csv_file)
        df_result = pd.read_csv('result.csv')
        df_constructor_result = df_result[['constructor_id', 'result_id']]

        # Write the DataFrame to the PostgreSQL table
        df_constructor_result.to_sql('constructor_result', con=engine, if_exists="append", index=False, method="multi", chunksize=100)
        # df.to_sql(table_name, con=engine, if_exists="append", index=False)
        print(f"Data from {csv_file} successfully imported into {table_name}!")
except Exception as e:
    print(f"An error occurred while importing {csv_file} into {table_name}: {e}")


### Query 1: List circuits in the USA

In [ ]:
USA_circuits= psql.read_sql("""
                            SELECT *
                            FROM circuit
                            WHERE country = 'USA'
                            """, engine)
display(USA_circuits.style)

### Query 2: List race count of the top 10 drivers in order


In [31]:
driver_race_counts= psql.read_sql("""
                                  SELECT * 
                                  FROM driver AS d
                                  JOIN 
                                    (SELECT driver_id, COUNT(race_id) AS race_count
                                    FROM result
                                    GROUP BY driver_id
                                    ) AS r
                                  ON d.driver_id = r.driver_id
                                  ORDER BY race_count DESC
                                  LIMIT 10
                                """, engine)
display(driver_race_counts.style)

,driver_id,code,first_name,last_name,dob,nationality,url,driver_id,race_count
0,4,ALO,Fernando,Alonso,1981-07-29,Spanish,http://en.wikipedia.org/wiki/Fernando_Alonso,4,392
1,8,RAI,Kimi,Rï¿½ikkï¿½nen,1979-10-17,Finnish,http://en.wikipedia.org/wiki/Kimi_R%C3%A4ikk%C3%B6nen,8,352
2,1,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,1,344
3,22,BAR,Rubens,Barrichello,1972-05-23,Brazilian,http://en.wikipedia.org/wiki/Rubens_Barrichello,22,326
4,18,BUT,Jenson,Button,1980-01-19,British,http://en.wikipedia.org/wiki/Jenson_Button,18,309
5,30,MSC,Michael,Schumacher,1969-01-03,German,http://en.wikipedia.org/wiki/Michael_Schumacher,30,308
6,20,VET,Sebastian,Vettel,1987-07-03,German,http://en.wikipedia.org/wiki/Sebastian_Vettel,20,300
7,13,MAS,Felipe,Massa,1981-04-25,Brazilian,http://en.wikipedia.org/wiki/Felipe_Massa,13,271
8,815,PER,Sergio,Pï¿½rez,1990-01-26,Mexican,http://en.wikipedia.org/wiki/Sergio_P%C3%A9rez,815,271
9,119,\N,Riccardo,Patrese,1954-04-17,Italian,http://en.wikipedia.org/wiki/Riccardo_Patrese,119,257


### Query 3: Which driver has the most wins?

In [47]:
driver_win_counts= psql.read_sql("""
                                  SELECT * 
                                  FROM driver AS d
                                  JOIN 
                                    (SELECT driver_id, COUNT(CASE WHEN rank = '1' THEN 1 END) AS first_place_count
                                    FROM result
                                    GROUP BY driver_id
                                    ) AS r
                                  ON d.driver_id = r.driver_id
                                  ORDER BY r.first_place_count DESC
                                  LIMIT 10
                                """, engine)
display(driver_win_counts.style)

,driver_id,code,first_name,last_name,dob,nationality,url,driver_id,first_place_count
0,1,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,1,66
1,8,RAI,Kimi,Rï¿½ikkï¿½nen,1979-10-17,Finnish,http://en.wikipedia.org/wiki/Kimi_R%C3%A4ikk%C3%B6nen,8,42
2,20,VET,Sebastian,Vettel,1987-07-03,German,http://en.wikipedia.org/wiki/Sebastian_Vettel,20,38
3,830,VER,Max,Verstappen,1997-09-30,Dutch,http://en.wikipedia.org/wiki/Max_Verstappen,830,32
4,4,ALO,Fernando,Alonso,1981-07-29,Spanish,http://en.wikipedia.org/wiki/Fernando_Alonso,4,25
5,30,MSC,Michael,Schumacher,1969-01-03,German,http://en.wikipedia.org/wiki/Michael_Schumacher,30,21
6,3,ROS,Nico,Rosberg,1985-06-27,German,http://en.wikipedia.org/wiki/Nico_Rosberg,3,20
7,822,BOT,Valtteri,Bottas,1989-08-28,Finnish,http://en.wikipedia.org/wiki/Valtteri_Bottas,822,19
8,17,WEB,Mark,Webber,1976-08-27,Australian,http://en.wikipedia.org/wiki/Mark_Webber_(racing_driver),17,19
9,817,RIC,Daniel,Ricciardo,1989-07-01,Australian,http://en.wikipedia.org/wiki/Daniel_Ricciardo,817,16
